In [ ]:
using Distributed
# addprocs(47)
ifelse(nprocs()<2, addprocs(2), nothing);

In [ ]:
nprocs()

# Paper effective model

In [ ]:
using LinearAlgebra
# using Plots
# pyplot()

In [ ]:
using Revise
using LatticeQM
using LatticeQM.Meanfield

In [ ]:
# Linear interpolation between the two parameter regimes
e0 = 1
t1(s) = e0 * (0.04)          * (1-s) + s * e0 * (0.06+0.05im)
t2(s) = e0 * (0.10 - 0.04im) * (1-s) + s * e0 * (0.06-0.05im)

In [ ]:
lat = Geometries2D.triangular_supercell()
slat = Structure.superlattice(lat, [[3,0] [0,3]])
Structure.foldPC!(slat; shift=[0.2,0.0,0.0])
# plot(slat, [0.0 for i in 1:Structure.countorbitals(slat)]; supercell=0, size=(200,200))

In [ ]:
# Set up lattice
lat0 = Geometries2D.triangular()
sz0 = getoperator(lat0, "sz")

lat = Geometries2D.triangular_supercell()
ks = kpath(lat, ["M2","Γ", "K", "M", "K'", "Γ", "M"]; num_points=500)
sz = getoperator(lat, "sz")
sub1, sub2, sub3 = [getoperator(lat, "sublattice", i, 2) for i=1:3]

slat = Structure.superlattice(lat, [[3,0] [0,3]])
Structure.foldPC!(slat; shift=[0.2,0.0,0.0])
ks2 = kpath(slat, ["M2","Γ", "K", "M", "K'", "Γ", "M"]; num_points=100)
sz2 = getoperator(slat, "sz")

nothing

In [ ]:
# Get nearest-neighbor hops in the honeycomb lattice
s = 0.0
hops0 = Operators.gethaldanelike(lat0; t1=t1(s), t2=t2(s))
hops = Operators.gethaldanelike(lat; t1=t1(s), t2=t2(s))
hops2 = Operators.gethaldanelike(slat; t1=t1(s), t2=t2(s))
nothing

In [ ]:
bands = getbands(hops0, ks, sz0; format=:dense)
# bands = getbands(hops2, ks2, sz2; format=:dense)
# p = plot(bands; markercolor=:PiYG, size=(400,130))
# p

In [ ]:
using ProgressMeter; ProgressMeter.ijulia_behavior(:clear);

In [ ]:
v = gethubbard(slat; mode=:σx, a=0.5, U=0.8) # interaction potential
ρ_init = 0.0 * initialguess(v, :ferro; lat=slat) + 1.0 * initialguess(v, :random; lat=slat) # initial guess

hf = hartreefock(hops2, v)

@time ρ_sol, ϵ_GS, HMF, converged, error = solveselfconsistent( # run the calculation
    hf, ρ_init, 0.5; klin=5, parallel=true, iterations=300, tol=1e-7,# p_norm=Inf,
    T=0.0001, β=0.30,  show_trace=true, clear_trace=true, verbose=false
)

# m1,m2,m3 = real.(Operators.magnetization(ρ_sol, [sub1,sub2,sub3], lat))
M = Operators.localmagnetization(ρ_sol, slat)
m = sum(M; dims=2)
Mabs = norm(m)
dens = Operators.density(ρ_sol)
@info("Groundstate energy", ϵ_GS)
@info("Magnetizations", Mabs, norm(M[:,1]), norm(M[:,2]), norm(M[:,3]))
@info("Magnetization vectors", Mabs, m, M)
@info("Density", dens)

# Get the bands without mean-field terms
ks = kpath(slat; num_points=200)
bands = getbands(hops2, ks, sz2)

# Get the bands with mean-field terms
bands_mf = getbands(HMF.h, ks, sz2)
bands_mf.bands .-= HMF.μ

###################################################################################################
###################################################################################################

using DelimitedFiles
XYZ = transpose(Structure.positions(slat))
mkpath("output"); writedlm("output/positions.out", XYZ)
mkpath("output"); writedlm("output/magnetization.out", M)


In [ ]:
let bands=bands, bands_mf=bands_mf
    using Plots
    
    p1 = plot(bands; markersize=2, size=(600,200))
    p2 = plot(bands_mf; markersize=2, size=(600,200))

    plot!(p1, title="non-interacting")
    plot!(p2, title="Hubbard meanfield")
    p = plot(p1,p2, titlefont=font(8))
    display(p)

    mkpath("output"); savefig("output/bandsmeanfield.pdf")
end

In [ ]:
let XYZ=XYZ, M=transpose(M)
    using PyPlot
    #pygui(true)
    using3D()

    fig = figure()
    ax = fig.gca(projection="3d")

    ax.scatter(eachcol(XYZ)...)
    ax.quiver(eachcol(XYZ)..., eachcol(M)..., length=0.5, pivot="middle", color="r")
    ax.quiver(sum(eachrow(XYZ))/3..., eachcol(M)..., length=0.15, pivot="tail", color="k")
    ax.set_xlim3d([0.0,1.0])
    ax.set_ylim3d([0.0,1.0])
    ax.set_zlim3d([-0.5,0.5])
    show()
end